<a href="https://colab.research.google.com/github/arbiterFF/Finance/blob/main/Optimal_portfolio_Sharpe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install numpy pandas pandas-datareader scipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
%pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [52]:
import numpy as np
import pandas as pd
import yfinance as yf
import datetime
from scipy.optimize import minimize

# Define the stocks in your portfolio
stocks = ['BHP.AX', 'BXB.AX', 'CSL.AX', 'MQG.AX', 'NAB.AX', 'RIO.AX', 'RMD.AX', 'SHL.AX', 'TCL.AX', 'WDS.AX', 'WTC.AX', 'TNE.AX', 'TLX.AX', 'APA.AX', 'TLS.AX']

# Set the time period
end_date = datetime.datetime.now()
start_date = end_date - datetime.timedelta(days=180)

# Fetch the stock price data
price_data = yf.download(stocks, start=start_date, end=end_date)['Adj Close']
price_data = price_data.reindex(sorted(price_data.columns), axis=1)

# Calculate daily returns
returns = price_data.pct_change().dropna()

def calculate_annualized_return(weights, mean_returns, cov_matrix):
    return np.sum(mean_returns * weights) * 252

def calculate_annualized_volatility(weights, cov_matrix):
    return np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)) * 252)

def neg_sharpe_ratio(weights, mean_returns, cov_matrix, risk_free_rate):
    portfolio_return = calculate_annualized_return(weights, mean_returns, cov_matrix)
    portfolio_volatility = calculate_annualized_volatility(weights, cov_matrix)
    return -(portfolio_return - risk_free_rate) / portfolio_volatility

def optimize_portfolio(weights, mean_returns, cov_matrix, risk_free_rate, min_weight):
    constraints = (
        {'type': 'eq', 'fun': lambda x: np.sum(x) - 1}
    )
    bounds = tuple((min_weight, 1) for _ in range(len(stocks)))
    bounds = tuple(bounds)
    result = minimize(neg_sharpe_ratio, weights, args=(mean_returns, cov_matrix, risk_free_rate), bounds=bounds, constraints=constraints)
    return result

def portfolio_performance(weights, mean_returns, cov_matrix, risk_free_rate):
    portfolio_return = np.sum(mean_returns * weights) * 252
    portfolio_std_dev = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)) * 252)
    sharpe_ratio = (portfolio_return - risk_free_rate) / portfolio_std_dev
    return portfolio_return, portfolio_std_dev, sharpe_ratio

# Calculate mean returns and covariance matrix
mean_returns = returns.mean()
cov_matrix = returns.cov()

# Print individual stock expected return and standard deviation
print("Individual stock performance:")
for stock in stocks:
    stock_mean_return = mean_returns[stock] * 252
    stock_std_dev = np.sqrt(cov_matrix.loc[stock, stock] *252)
    print(f"{stock}: Expected annual return: {stock_mean_return * 100:.2f}%, Annualized volatility: {stock_std_dev * 100:.2f}%")

# Set the initial weights and risk-free rate
initial_weights = np.array([1 / len(stocks)] * len(stocks))

risk_free_rate = 0.04
min_weight=0.2/len(stocks)
print(f"Minimum weight for each stock: {min_weight:.4f}")

# Optimize the portfolio
result = optimize_portfolio(initial_weights, mean_returns, cov_matrix, risk_free_rate, min_weight)

# Display the optimized portfolio weights
optimized_weights = result.x
print("Optimized portfolio weights:")
for stock, weight in zip(stocks, optimized_weights):
    print(f"{stock}: {weight:.4f}")

# Calculate and display portfolio performance
portfolio_return, portfolio_std_dev, sharpe_ratio = portfolio_performance(optimized_weights, mean_returns, cov_matrix, risk_free_rate)

print ("E(r) = ", portfolio_return*100, "%")
print ("sigma = ", portfolio_std_dev*100, "%")
print ("Sharpe = ", sharpe_ratio)

[*********************100%***********************]  14 of 14 completed
Individual stock performance:
BHP.AX: Expected annual return: 41.19%, Annualized volatility: 27.40%
BXB.AX: Expected annual return: 33.16%, Annualized volatility: 18.23%
CSL.AX: Expected annual return: 0.94%, Annualized volatility: 16.70%
MQG.AX: Expected annual return: 23.21%, Annualized volatility: 25.05%
NAB.AX: Expected annual return: -12.92%, Annualized volatility: 17.61%
RIO.AX: Expected annual return: 50.98%, Annualized volatility: 26.43%
RMD.AX: Expected annual return: -15.77%, Annualized volatility: 24.81%
SHL.AX: Expected annual return: 24.94%, Annualized volatility: 29.20%
TCL.AX: Expected annual return: 25.69%, Annualized volatility: 15.16%
WDS.AX: Expected annual return: 13.79%, Annualized volatility: 31.59%
WTC.AX: Expected annual return: 30.85%, Annualized volatility: 39.04%
TNE.AX: Expected annual return: 53.70%, Annualized volatility: 26.59%
TLX.AX: Expected annual return: 56.64%, Annualized volatil